# Measuring the physical performance of players using tracking data

We will accomplish the following objectives in this notebook:

1. Create and save a movie clip of the goal we analysed in the last notebook. This is to show how useful and important such movie clips are in football analytics.


2. Calculate and plot player velocities.


3. Create physical performance reports for the home team players which would include information such as minutes played, total distance covered, distance covered in different activities (like walking, jogging, running, sprinting), and total number of sprints made.


4. Plot all the sustained sprints of a particular home team player.

### Import libraries and modules

In [1]:
import modules.Metrica_IO as mio # module for input output operations involving Metrica Sports data
import modules.Metrica_Viz as mviz # module for visualising Metrica Sports data
import modules.Metrica_Velocities as mvel # module for measuring player velocities, smoothed using a Savitzky-Golay filter, with Metrica Sports data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Video # used to display videos in Jupyter Notebooks

### Initial setup

In [2]:
DATADIR = 'LaurieOnTracking-master/sample-data-master/data/' # setting up the path to data
game_id = 1 # initialising the game id (we will be analysing the game 1's data; Metrica Sports has provided data for 2 games)

### Read in the event and the tracking data

In [3]:
events = mio.read_event_data(DATADIR,game_id) # reading in the event data
tracking_home = mio.tracking_data(DATADIR,game_id,'Home') # reading in the tracking data for the home team
tracking_away = mio.tracking_data(DATADIR,game_id,'Away') # reading in the tracking data for the away team

Reading team: home
Reading team: away


In [4]:
events.head(3)

,Team,Type,Subtype,Period,Start Frame,Start Time [s],End Frame,End Time [s],From,To,Start X,Start Y,End X,End Y
0,Away,SET PIECE,KICK OFF,1,1,0.04,0,0.00,Player19,NaN,NaN,NaN,NaN,NaN
1,Away,PASS,NaN,1,1,0.04,3,0.12,Player19,Player21,0.45,0.39,0.55,0.43
2,Away,PASS,NaN,1,3,0.12,17,0.68,Player21,Player15,0.55,0.43,0.58,0.21


In [5]:
tracking_home.head(3)

,Period,Time [s],Home_11_x,Home_11_y,Home_1_x,Home_1_y,Home_2_x,Home_2_y,Home_3_x,Home_3_y,...,Home_10_x,Home_10_y,Home_12_x,Home_12_y,Home_13_x,Home_13_y,Home_14_x,Home_14_y,ball_x,ball_y
Frame,,,,,,,,,,,,,,,,,,,,,
1,1,0.04,0.00082,0.48238,0.32648,0.65322,0.33701,0.48863,0.30927,0.35529,...,0.55243,0.43269,NaN,NaN,NaN,NaN,NaN,NaN,0.45472,0.38709
2,1,0.08,0.00096,0.48238,0.32648,0.65322,0.33701,0.48863,0.30927,0.35529,...,0.55243,0.43269,NaN,NaN,NaN,NaN,NaN,NaN,0.49645,0.40656
3,1,0.12,0.00114,0.48238,0.32648,0.65322,0.33701,0.48863,0.30927,0.35529,...,0.55243,0.43269,NaN,NaN,NaN,NaN,NaN,NaN,0.53716,0.42556


In [6]:
tracking_away.head(3)

,Period,Time [s],Away_25_x,Away_25_y,Away_15_x,Away_15_y,Away_16_x,Away_16_y,Away_17_x,Away_17_y,...,Away_24_x,Away_24_y,Away_26_x,Away_26_y,Away_27_x,Away_27_y,Away_28_x,Away_28_y,ball_x,ball_y
Frame,,,,,,,,,,,,,,,,,,,,,
1,1,0.04,0.90509,0.47462,0.58393,0.20794,0.67658,0.4671,0.6731,0.76476,...,0.37833,0.27383,NaN,NaN,NaN,NaN,NaN,NaN,0.45472,0.38709
2,1,0.08,0.90494,0.47462,0.58393,0.20794,0.67658,0.4671,0.6731,0.76476,...,0.37833,0.27383,NaN,NaN,NaN,NaN,NaN,NaN,0.49645,0.40656
3,1,0.12,0.90434,0.47463,0.58393,0.20794,0.67658,0.4671,0.6731,0.76476,...,0.37833,0.27383,NaN,NaN,NaN,NaN,NaN,NaN,0.53716,0.42556


### Data preprocessing

In [7]:
# converting the units of player and ball position coordinates from Metrica Sports units to metres
events = mio.to_metric_coordinates(events)
tracking_home = mio.to_metric_coordinates(tracking_home)
tracking_away = mio.to_metric_coordinates(tracking_away)

In [8]:
# this is for future use, if needed

# reversing the direction of play in the second half so that home team is always attacking from right->left
# tracking_home, tracking_away, events = mio.to_single_playing_direction(tracking_home, tracking_away, events)

### Data analysis and visualisation

Let's accomplish our objectives one-by-one:

__1. Create and save a movie clip of the goal we analysed in the last notebook.__

For this purpose, we need the frame numbers of the events when the passing movement started (which eventually led to the goal) as well as when the goal was scored. Let's get those first using the information we found in the last notebook.

*NOTE: In the code cell below, I am adding 100 frames to goal_frame because the goal_frame value itself denotes the instant when the goalscorer just hits the ball. Therefore, using just the goal_frame value will not show us the movement of the ball into the goal after being shot. After a few trails, I have figured out that adding the next 100 frames after the goal_frame value helps capture the complete goal scoring event.*

In [9]:
pass_move_start_index = 1111
goal_index = 1114
pass_move_start_frame = events.loc[pass_move_start_index]['Start Frame'] # saving the frame number of the event in which the passing movement started
goal_frame = events.loc[goal_index]['Start Frame'] + 100 # saving the frame number of the event in which the goal was scored

Now, let's create the movie clip for this goal and the passing movements involved and save it to the directory where all of our Metrica Sports data is stored.

In [12]:
mviz.save_match_clip(tracking_home.iloc[pass_move_start_frame:goal_frame],
                     tracking_away.iloc[pass_move_start_frame:goal_frame],
                     fpath='images and videos/',
                     fname='game_1_goal_2')

Generating movie...

D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:207: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team['ball_x'], team['ball_y'], 'ko', MarkerSize=6, alpha=1.0, LineWidth=0)
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarn

D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:207: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team['ball_x'], team['ball_y'], 'ko', MarkerSize=6, alpha=1.0, LineWidth=0)
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarn

D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:207: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team['ball_x'], team['ball_y'], 'ko', MarkerSize=6, alpha=1.0, LineWidth=0)
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarn

D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:207: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team['ball_x'], team['ball_y'], 'ko', MarkerSize=6, alpha=1.0, LineWidth=0)
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarn

...

D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:207: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team['ball_x'], team['ball_y'], 'ko', MarkerSize=6, alpha=1.0, LineWidth=0)
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarn

D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:207: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team['ball_x'], team['ball_y'], 'ko', MarkerSize=6, alpha=1.0, LineWidth=0)
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarn

D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:207: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team['ball_x'], team['ball_y'], 'ko', MarkerSize=6, alpha=1.0, LineWidth=0)
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarn

D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
D:\friends-of-tracking\modules\Metrica_Viz.py:207: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  objs, = ax.plot( team['ball_x'], team['ball_y'], 'ko', MarkerSize=6, alpha=1.0, LineWidth=0)
D:\friends-of-tracking\modules\Metrica_Viz.py:199: MatplotlibDeprecationWarn

done


Let's have a look at the movie clip:

In [13]:
movie_clip_path = 'images and videos/game_1_goal_2.mp4'
Video(movie_clip_path, width=800, height=400)

Just like we observed in the last notebook, here we can see that the goalscorer made a really good run between the opposition defenders. The player who provided the assist also waited and released the ball at the right time.

__NOTE:__
The video isn't being displayed when the notebook is uploaded to GitHub. I am working on this. Meanwhile, you can download the movie clip from [here](images%20and%20videos/game_1_goal_2.mp4). Just go to this link and save the video to your local system. If you download/clone the repository to your local system, the video will be playable in this Jupyter Notebook. Also, if you know any solution to this issue, please let me know. Thanks!

In [19]:
from IPython.display import HTML

HTML('<iframe width="600" height="400" src="https://www.youtube.com/embed/zjNajrkqmww" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

In [24]:
%%HTML

<video>
  <source src="https://www.youtube.com/embed/zjNajrkqmww" type="video/mp4">
</video>

# %%HTML
# <iframe width="600" height="400" src="https://www.youtube.com/embed/zjNajrkqmww" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

__2. Calculate and plot player velocities.__